In [ ]:
import healpy as hp
%matplotlib inline
rc('figure', figsize=(16, 8))
rc('font', size=15)

# %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [ ]:
### Variance maps in muK2
varI2048 = hp.read_map('HFI_SkyMap_217_2048_R3.01_full.fits', field=4) * 1e12
varQ2048 = hp.read_map('HFI_SkyMap_217_2048_R3.01_full.fits', field=7) * 1e12
varU2048 = hp.read_map('HFI_SkyMap_217_2048_R3.01_full.fits', field=9) * 1e12
print(np.shape(varI2048))

In [ ]:
hp.mollview(varI2048, sub=(1,3,1), cmap='jet')
hp.mollview(varQ2048, sub=(1,3,2), cmap='jet')
hp.mollview(varU2048, sub=(1,3,3), cmap='jet')

In [ ]:
### Degrade the maps to 256
nsout = 256
varI256 = hp.ud_grade(varI2048, nside_out=256, power=2)
hp.mollview(varI2048, sub=(1,2,1), title='2048', cmap='jet')
hp.mollview(varI256, sub=(1,2,2), title='256', cmap='jet')


In [ ]:
nbmc = 100
allmaps = np.zeros((nbmc, 12*nsout**2))

nsin = 2048
for i in range(nbmc):
    if (i%10)==0:
        print(i)
    mm = np.random.randn(12*nsin**2) * np.sqrt(varI2048)
    allmaps[i,:] = hp.ud_grade(mm, nside_out=nsout)


In [ ]:
empvar = np.var(allmaps, axis=0)
hp.mollview(varI256, sub=(2,2,1), title='Var 256', cmap='jet')
hp.mollview(empvar, sub=(2,2,2), title='Emprical Var 256', cmap='jet')
hp.mollview(empvar/varI256, sub=(2,2,3), title='Ratio', cmap='jet')
subplot(2,2,4)
plot(varI256, empvar, ',')
plot(np.arange(50), 'k--')
xlabel('Theoretical Variance ns=256')
ylabel('Empirical MC ns=256')

# SImulation of the beam effect

In [ ]:
fwhm_q_150 = 0.39268176 # degrees
nsout = 256
nbmc = 300
allmaps = np.zeros((nbmc, 12*nsout**2))

nsin = 2048
for i in range(nbmc):
    if (i%10)==0:
        print(i)
    mm = np.random.randn(12*nsout**2) * np.sqrt(varI256)
    allmaps[i,:] = hp.smoothing(mm, fwhm=np.radians(fwhm_q_150))
    

In [ ]:
from qubic import fibtools as ft
newvarmap = np.var(allmaps, axis=0)
hp.mollview(newvarmap, sub=(1,2,1), title='Empirical Variance after smoothing')
subplot(1,2,2)
plot(varI256, newvarmap, 'k,')
ft.profile(varI256, newvarmap, nbins=10, plot=True)
xlabel('Unsmoothed Variance')
ylabel('Empirical Variance after smoothing')

## Calculation of theoretical variance after smoothing:
One has an image $M_{i}$ with variance $v_{i}$. It is smoothed with a normed Gaussian kernel $k_{ij}$:
$$M'_{i} = \sum_j k_{ij} M_j$$
Variances add linearly, therefore:
$
\begin{align}
Var(M'_i) &= \sum_j Var(k_{ij} M_j) \\
&= \sum_j k_{ij}^2 Var(M_j)
\end{align}
$
where the kernel is:
$\begin{align}
k(\vec{n}-\vec{n}_0) &\equiv G(\vec{n}-\vec{n}_0,\sigma)\\ 
&=\frac{1}{2\pi\sigma^2}\exp\left( -\frac{\left| \vec{n}-\vec{n}_0\right|^2}{2\sigma^2}\right)
\end{align}
$
Therefore:
$\begin{align}
k^2(\vec{n}-\vec{n}_0) 
&=\left(\frac{1}{2\pi\sigma^2}\right)^2\exp\left( -\frac{\left| \vec{n}-\vec{n}_0\right|^2}{\sigma^2}\right) \\
&= \frac{1}{2\pi\sigma^2} \times \frac{1}{2\pi\sigma^2} \times \exp\left( -\frac{\left| \vec{n}-\vec{n}_0\right|^2}{2\left(\sigma/\sqrt{2}\right)^2}\right) \\
&= \frac{1}{2\pi\sigma^2} \times \frac{1}{2\times 2\pi\left(\sigma/\sqrt{2}\right)^2} \times \exp\left( -\frac{\left| \vec{n}-\vec{n}_0\right|^2}{2\left(\sigma/\sqrt{2}\right)^2}\right) \\
&= \frac{1}{4\pi\sigma^2}\times  G\left(\vec{n}-\vec{n}_0,\sigma/\sqrt{2}\right)
\end{align}
$
Where $\sigma$ is given in units of pixel size.

In [ ]:
factor = (4*np.pi*(fwhm_q_150/2.35/np.degrees(hp.nside2resol(nsout)))**2)
varnew = hp.smoothing(varI256, fwhm=np.radians(fwhm_q_150/sqrt(2))) / factor


hp.mollview(newvarmap, sub=(2,3,1), cmap='jet', title='Empirical Variance After smoothing')
hp.mollview(varnew, sub=(2,3,2), cmap='jet', title='Theoretical Variance after smoothing')
hp.mollview(newvarmap/varnew, sub=(2,3,3), title='Ratio', cmap='jet')
subplot(2,2,3)
plot(newvarmap, varnew, ',')
plot(np.arange(np.max(varnew)), 'k--')
ft.profile(newvarmap, varnew, nbins=30, plot=True)
xlabel('Theoretical Variance after smmothing ns=256')
ylabel('Empirical MC ns=256')
subplot(2,2,4)
a=hist(newvarmap/varnew, bins=100, range=[0.5,1.5])
axvline(x=1,ls='--', color='k')
xlabel('Ratio')

##### 

In [ ]:
np.degrees(hp.nside2resol(256))